In [32]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [33]:
#  TODO:
"""
posts.    [profile_id]   ==  accounts.              [id]
accounts. [_id]          ==  sources_for_followers. [_id]


In accounts we can change ["id"] to ["profile_id"]
In posts    we can change ["id"] to ["post_id"]


+ 3 <alt_id> from ACCOUNTS (maybe different post tag) == 3 <profile_id> from POSTS
Need to change with np.where <profile_id> in POSTS to <alt_id> because values not matched
"""

'\nposts.    [profile_id]   ==  accounts.              [id]\naccounts. [_id]          ==  sources_for_followers. [_id]\n\n\nIn accounts we can change ["id"] to ["profile_id"]\nIn posts    we can change ["id"] to ["post_id"]\n\n\n+ 3 <alt_id> from ACCOUNTS (maybe different post tag) == 3 <profile_id> from POSTS\nNeed to change with np.where <profile_id> in POSTS to <alt_id> because values not matched\n'

In [34]:
# Define your PostgreSQL connection parameters
db_params = {
    'dbname': 'mydb',
    'user': 'postgres',
    'password': '1',
    'host': 'localhost',
    'port': '5432'
}

In [35]:
# TODO : Rename columns with psycopg2
conn = psycopg2.connect(
    dbname=db_params['dbname'],
    user=db_params['user'],
    password=db_params['password'],
    host=db_params['host'],
    port=db_params['port']
)

# SQL command to rename columns
rename_commands = [
    """ALTER TABLE test_task.accounts RENAME COLUMN id TO profile_id;""",
    """ALTER TABLE test_task.posts RENAME COLUMN id TO post_id;"""
]

"""with conn:
    with conn.cursor() as cursor:
        for command in rename_commands:
            cursor.execute(command)"""

conn.close()

In [36]:
# TODO : Extract data from Postgres sqlalchemy

connection_url = f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['dbname']}"

engine = create_engine(connection_url)


sql_query_allData = f"""
WITH tj AS (
    SELECT a.*, s.* -- Ensure that profile_id and other columns are selected
    FROM test_task.accounts a
    JOIN test_task.sources_for_followers s ON a._id = s._id
),

df1 AS (
    SELECT tj.*, p1.*
    FROM tj
    LEFT JOIN test_task.posts p1 ON tj.profile_id = p1.profile_id
),

df2 AS (
    SELECT tj.*, p2.*
    FROM tj
    LEFT JOIN test_task.posts p2 ON tj.id_alt = p2.profile_id
)

SELECT * FROM df1
UNION ALL  
SELECT * FROM df2;

"""


data = pd.read_sql(sql_query_allData, engine)


# Close the connection
engine.dispose()

In [37]:
data = data[~data['post_id'].isna()]
data = data.groupby(['username'])[['comments_count']].agg(['sum','count'])


In [38]:
data

comments_count      
                                                 sum count
username                                                  
1tvnews                                         20.0    39
4media.INFO                                      1.0    39
NatoAustritt                                    30.0    39
NuoViso.TV                                      11.0    39
Prüfe-alles-glaube-wenig-denke-selbst         2663.0    39
RT.InoTV                                         3.0    39
RussophiliaBulgarophilia                        44.0    39
Sergiu.Mihalcea.PaginaOficiala                1269.0    39
ambrusitalia                                   384.0    39
armadaRF                                       189.0    39
friedentotal                                   656.0    39
george.simion.unire                           1623.0    39
giorgio.bianchi.100                            266.0    39
group/rumenradevspasitel                       400.0    39
ing.milan.uhrik                                  4.0    39
lentaru                                          0.0     7
mod.mil.rus                                      6.0    39
neverforgetniki                                 87.0    39
nicvoiculet                                     14.0    39
russembangola                                  662.0    39
shesti.elena                                    46.0    39
sputnik.france                                  30.0    39
tassagency                                      18.0    39
tsoncho.ganev                                  145.0    39
ukraina.ru                                     150.0    39
unzensuriert                                  3853.0    39
wimacht                                          5.0    39
wochenblick                                      8.0    39